# Nuke button

In [ ]:
# Delete all files and folders in the working directory. Use with caution!
import shutil
shutil.rmtree("/kaggle/working", ignore_errors=True)

# Clone repo

In [ ]:
%cd /kaggle/working
!git clone https://github.com/BinhPQ2/manga_read_along.git
# !git clone -b magiv2 https://github.com/BinhPQ2/manga_read_along.git

In [ ]:
%cd /kaggle/working/manga_read_along

raw_image_path = "input/raw"
character_path = "input/character"

raw_image_rename_path = "output/renamed"
colorized_path = "output/colorized"
json_path = "output/json"
transcript_path = "output/transcript"
audio_path = "output/audio"
final_output_path ="output/output_final"

In [ ]:
!git submodule init
!git submodule update --remote

In [3]:
cat /kaggle/working/manga_read_along/magi_functional/requirements_kaggle.txt | xargs -n 1 pip install -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.3 requires cubinlinker, which is not installed.
cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.3 requires ptxcompiler, which is not installed.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
ucxx 0.39.1 requires libucx>=1.15.0, which is not installed.
albucore 0.0.17 requires numpy>=1.24, but you have numpy 1.22.0 which is incompatible.
albumentations 1.4.17 requires numpy>=1.24.4, but you have numpy 1.22.0 which is incompatible.
apache-beam 2.46.0 requires cloudpickle~=2.2.1, but you have cloudpickle 3.0.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you h

In [ ]:
from PIL import Image
import numpy as np
from transformers import AutoModel
import torch
import os
import random
import re
import shutil
import argparse
import torch
from TTS.api import TTS
from unittest.mock import patch

Check GPU and set device

In [4]:
import torch
import os

if torch.cuda.is_available():
    !nvidia-smi
    !nvcc --version
else:
    print("GPU is not available")


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Running on {device}")

Wed Nov  6 18:09:33 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P0             25W /  250W |       3MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Initialize model

## Download MAGI model

In [5]:
magiv2 = AutoModel.from_pretrained("ragavsachdeva/magiv2", trust_remote_code=True).to(device).eval()

config.json:   0%|          | 0.00/13.4k [00:00<?, ?B/s]

configuration_magiv2.py:   0%|          | 0.00/1.70k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ragavsachdeva/magiv2:
- configuration_magiv2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modelling_magiv2.py:   0%|          | 0.00/34.0k [00:00<?, ?B/s]

utils.py:   0%|          | 0.00/16.4k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
VisionEncoderDecoderModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

## Download weight for colorization

In [7]:
!gdown 1qmxUEKADkEM4iYLp1fpPLLKnfZ6tcF-t -O /kaggle/working/manga_read_along/manga-colorization-v2-custom/networks/generator.zip
!mkdir /kaggle/working/manga_read_along/manga-colorization-v2-custom/denoising/models
!gdown 161oyQcYpdkVdw8gKz_MA8RD-Wtg9XDp3 -O /kaggle/working/manga_read_along/manga-colorization-v2-custom/denoising/models/net_rgb.pth

Downloading...
From (original): https://drive.google.com/uc?id=1qmxUEKADkEM4iYLp1fpPLLKnfZ6tcF-t
From (redirected): https://drive.google.com/uc?id=1qmxUEKADkEM4iYLp1fpPLLKnfZ6tcF-t&confirm=t&uuid=b8e27ce6-40aa-4a91-a04e-4dae4bb50a03
To: /kaggle/working/manga_read_along/manga-colorization-v2-custom/networks/generator.zip
100%|████████████████████████████████████████| 129M/129M [00:03<00:00, 35.4MB/s]
mkdir: cannot create directory '/kaggle/working/manga_read_along/manga-colorization-v2-custom/denoising/models': File exists
Downloading...
From: https://drive.google.com/uc?id=161oyQcYpdkVdw8gKz_MA8RD-Wtg9XDp3
To: /kaggle/working/manga_read_along/manga-colorization-v2-custom/denoising/models/net_rgb.pth
100%|███████████████████████████████████████| 3.44M/3.44M [00:00<00:00, 211MB/s]


## Download weight for Text-to-Speech model

In [10]:


# Mock input to automatically respond with 'y'
with patch('builtins.input', return_value='y'):
    tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)

 > You must confirm the following:
 | > "I have purchased a commercial license from Coqui: licensing@coqui.ai"
 | > "Otherwise, I agree to the terms of the non-commercial CPML: https://coqui.ai/cpml" - [y/n]
 > Downloading model to /root/.local/share/tts/tts_models--multilingual--multi-dataset--xtts_v2


100%|█████████▉| 1.87G/1.87G [00:44<00:00, 42.8MiB/s]
100%|██████████| 1.87G/1.87G [00:44<00:00, 41.7MiB/s]
100%|██████████| 4.37k/4.37k [00:00<00:00, 9.86kiB/s]

100%|██████████| 361k/361k [00:00<00:00, 459kiB/s]
100%|██████████| 32.0/32.0 [00:00<00:00, 49.9iB/s]
 45%|████▍     | 3.49M/7.75M [00:00<00:00, 34.9MiB/s]

 > Model's license - CPML
 > Check https://coqui.ai/cpml.txt for more info.
 > Using model: xtts


/opt/conda/lib/python3.10/site-packages/TTS/tts/layers/xtts/xtts_manager.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.speakers = torch.load(speaker_file_path)
10

# Magiv2

In [8]:
# Test module magiv2 (worked)
!python /kaggle/working/manga_read_along/magi_functional/magiv2.py --image {raw_image_path} --rename_image {raw_image_rename_path} --character {character_path} --json {json_path} --transcript {transcript_path}

Running on cuda
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
VisionEncoderDecoderModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture cod

# Colorization

In [11]:
!python "/kaggle/working/manga_read_along/manga-colorization-v2-custom/inference_v2.py" -p {raw_image_rename_path} -des_path "/kaggle/working/manga_read_along/manga-colorization-v2-custom/denoising/models/net_rgb.pth" -gen "/kaggle/working/manga_read_along/manga-colorization-v2-custom/networks/generator.zip" -s {colorized_path} -ds 0 --gpu

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Processing: output/renamed/001.jpg
Saved colorized image to: output/colorized/001.jpg
Processing: output/renamed/002.jpg
Saved colorized image to: output/colorized/002.jpg
Processing: output/renamed/000.jpg
Saved colorized image to: output/colorized/000.jpg


# Text to Speech

## Download Text-To-Speech model (library of TTS model is conflicted with Magi model, run this after run Magi for now)

In [13]:
voice_bank = "/kaggle/input/ravdess-emotional-speech-audio/"
male_character = ['teacher']
transcript_file = f"{transcript_path}/transcript.txt"

In [14]:
!python /kaggle/working/manga_read_along/magi_functional/text_to_speech.py -i {raw_image_rename_path} -v {voice_bank} -t {transcript_file} -o {audio_path}  -m male_character 

Running on cuda
 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts
/opt/conda/lib/python3.10/site-packages/TTS/tts/layers/xtts/xtts_manager.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on G

# Combine to video

In [16]:
!python /kaggle/working/manga_read_along/magi_functional/main.py -i {colorized_path} -j {json_path} -a {audio_path} -s {final_output_path} 

Deleted images
Processed and saved modified panel view images for: 000.jpg
Processed and saved modified panel view images for: 001.jpg
Processed and saved modified panel view images for: 002.jpg
Video saved as: output/output_final/video_Padding_True.mp4
Audio file not found for image: page_000_panel_000_bubble_000.jpg. Appending silence for 1 seconds.
Audio file not found for image: page_001_panel_000_bubble_000.jpg. Appending silence for 1 seconds.
Audio file not found for image: page_001_panel_001_bubble_000.jpg. Appending silence for 1 seconds.
Audio file not found for image: page_001_panel_002_bubble_000.jpg. Appending silence for 1 seconds.
Audio file not found for image: page_001_panel_003_bubble_000.jpg. Appending silence for 1 seconds.
Audio file not found for image: page_002_panel_000_bubble_000.jpg. Appending silence for 1 seconds.
Audio file not found for image: page_002_panel_001_bubble_000.jpg. Appending silence for 1 seconds.
Audio file not found for image: page_002_panel

# Download contents

## Download separate files

### Download json

In [ ]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(download_file_name, source_path):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip -rj {zip_name} {source_path}"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))

download_file("json_results", f"{json_output_dir}/*.json")

### Download image results

In [ ]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(download_file_name, source_path):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip -rj {zip_name} {source_path}"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))

download_file("image_results", result_image_output_dir)

### Download Audio

In [ ]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(download_file_name, source_path):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip -rj {zip_name} {source_path}"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))

download_file("audio_results", "/kaggle/working/manga_read_along/test/audio_output")

In [ ]:
!rm -rf /kaggle/working/audio_results.zip

### Download transcript

In [ ]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(download_file_name, source_path):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip -rj {zip_name} {source_path}"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))
download_file("transcript", "/kaggle/working/result/transcript.txt")

## Download all

In [ ]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(download_file_name, source_path):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip -r {zip_name} {source_path}"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))
    

!cp -r result_dir /
download_file("result", result_dir)

## Edit kaggle

In [ ]:
process_file = '''
TTS==0.22.0
einops==0.8.0
PuLP==2.9.0
Pillow==9.5.0
gdown==5.2.0
pydub==0.25.1
numpy==1.26.4
pandas==2.2.2
'''

with open("/kaggle/working/manga_read_along/magi_functional/requirements_kaggle.txt", "w") as file:
    file.write(process_file)
    print("Done")

# Experienmental notebook

## Magiv2

#### Create raw and character/names list (should apply rename first dumbass, TODO)

In [ ]:
import os
import re
from pathlib import Path

def create_chapter_pages_and_character_bank(manga_folder, character_folder):
    # Create lists for chapter pages and character bank
    chapter_pages = []
    character_bank = {
        "images": [],
        "names": []
    }

#     Iterate through manga images to create chapter_pages
    for image_file in os.listdir(manga_folder):
        if image_file.endswith(('.png', '.jpg', '.jpeg')):  # Check for image file extensions
            # Extract the page number using regex
            match = re.search(r'p(\d+)', image_file)
            if match:
                page_number = int(match.group(1))  # Convert to integer for sorting
                chapter_pages.append((page_number, image_file))  # Store as tuple (page_number, image_file)
            else:
                page_number = image_file.rsplit(".", 1)[0]
                chapter_pages.append((page_number, image_file))  # Store as tuple (page_number, image_file)

    # Sort chapter pages by page number
    chapter_pages.sort(key=lambda x: x[0])
    chapter_pages = [os.path.join(manga_folder, img[1]) for img in chapter_pages]  # Extract just the filenames after sorting

    # Iterate through character images to create character bank
    for char_image_file in os.listdir(character_folder):
        if char_image_file.endswith(('.png', '.jpg', '.jpeg')):  # Check for image file extensions
            # Split the filename to extract character name
            char_name = char_image_file.split('_')[0]  # Get the part before the underscore
            character_bank["images"].append(os.path.join(character_folder, char_image_file))
            character_bank["names"].append(char_name)
    return chapter_pages, character_bank

# Define your folders
# One Piece
# manga_folder = Path("/kaggle/working/magi_functional/data_test/personal_data/One_Piece/raw_manga")
# character_folder = Path("/kaggle/working/magi_functional/data_test/personal_data/One_Piece/character")

# Ruri Dragon
manga_folder = Path("/kaggle/working/manga_read_along/magi_functional/data_test/personal_data/Ruri_Dragon/raw")
character_folder = Path("/kaggle/working/manga_read_along/magi_functional/data_test/personal_data/Ruri_Dragon/character")

# Get chapter pages and character bank
chapter_pages_original, character_bank_original = create_chapter_pages_and_character_bank(manga_folder, character_folder)

chapter_pages_test = chapter_pages_original
character_bank_test = character_bank_original

# Print the results (for debugging)
print("Chapter Pages:")
print(chapter_pages_test)

print("\nCharacter Bank:")
print(character_bank_test)

### Process (OCR → Transcript)

In [ ]:
import os
import json

def read_image(path_to_image):
    with open(path_to_image, "rb") as file:
        image = Image.open(file).convert("L").convert("RGB")
        image = np.array(image)
    return image


chapter_pages = [read_image(x) for x in chapter_pages_test]
character_bank = character_bank_test.copy()
character_bank["images"] = [read_image(x) for x in character_bank_test["images"]]

with torch.no_grad():
    per_page_results = magiv2.do_chapter_wide_prediction(chapter_pages, character_bank, use_tqdm=True, do_ocr=True)

print("Continue with next cell")

### Save transcript and json file

In [ ]:
result_dir = "/kaggle/working/result"
json_output_dir = f"{result_dir}/json_results"
result_image_output_dir = f"{result_dir}/image_results"
transcript_output_dir = f"{result_dir}/transcript.txt"
os.makedirs(json_output_dir, exist_ok=True)  # Create the directory if it doesn't exist  
os.makedirs(result_image_output_dir, exist_ok=True)  # Create the directory if it doesn't exist

transcript = []
for i, (image, page_result) in enumerate(zip(chapter_pages, per_page_results)):
    image_name_ext = os.path.basename(chapter_pages_test[i]) 
    # Split the image name and its extension
    image_name, image_extension = os.path.splitext(image_name_ext)
    
#     model.visualise_single_image_prediction(image, page_result, os.path.join(result_image_output_dir, f"{image_name}.png")) # enable this if you want to see the result image included with all the annotation box
    # Save page_result to JSON
    json_file_path = os.path.join(json_output_dir, f"{image_name}.json")
    with open(json_file_path, 'w') as json_file:
        json.dump(page_result, json_file, indent=4)

    speaker_name = {
        text_idx: page_result["character_names"][char_idx] for text_idx, char_idx in page_result["text_character_associations"]
    }
    
    transcript.append(f"<page>{image_name}<endpage>")
    for j in range(len(page_result["ocr"])):
        if not page_result["is_essential_text"][j]:
            continue
        name = speaker_name.get(j, "unsure") 
        transcript.append(f"<name>{name}<endname>: {page_result['ocr'][j]}")
with open(transcript_output_dir, "w") as fh:
    for line in transcript:
        fh.write(line + "\n")

print("\n\nDone you WEEEEB!")

## Text to Speech

In [ ]:
voice_mapping = {
'other': "/kaggle/input/ravdess-emotional-speech-audio/Actor_02/03-01-01-01-01-01-02.wav",
'ruri': "/kaggle/input/ravdess-emotional-speech-audio/Actor_04/03-01-01-01-01-01-04.wav",
'teacher': "/kaggle/input/ravdess-emotional-speech-audio/Actor_01/03-01-01-01-01-01-01.wav",
'mom': "/kaggle/input/ravdess-emotional-speech-audio/Actor_08/03-01-01-01-01-01-08.wav",
'unsure': "/kaggle/input/ravdess-emotional-speech-audio/Actor_10/03-01-01-01-01-01-10.wav",
'ukka': "/kaggle/input/ravdess-emotional-speech-audio/Actor_12/03-01-01-01-01-01-12.wav"
}
 

In [ ]:
from magi_functional.utils.utils import rename_image_to_correct_format, get_digit_number_for_name_format, generate_name_format

images_folder = '/kaggle/working/manga_read_along/input/colorized'

number_of_digit_for_name = get_digit_number_for_name_format(images_folder)
name_format = generate_name_format(number_of_digit_for_name)
print(name_format)

In [ ]:
# Function to parse the transcript from a text file
def parse_transcript(transcript_path, line_end: int = None):
    pages = []
    current_page = None

    with open(transcript_path, 'r') as file:
        content = file.readlines()
        
    lines_to_process = content[:line_end] if line_end is not None else content
    
    for line in lines_to_process:
        if line.startswith("<page>"):
            # Start a new page
            if current_page is not None:
                pages.append(current_page)
            current_page = {"page": re.search(r'<page>(\d+)<endpage>', line).group(1), "lines": []}      
        elif line.startswith("<name>") and current_page is not None:
            # Extract character name and dialogue
            match = re.match(r"<name>([^<]+)<endname>:\s*(.+)", line)
            if match:
                character = match.group(1).lower()  # Lowercase for consistency
                dialogue = match.group(2)
                current_page["lines"].append((character, dialogue))

    # Add the last page if it exists
    if current_page is not None:
        pages.append(current_page)
        
    return pages

# Function to get all voice files from the specified directory
def get_voice_files(directory):
    all_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".wav"):
                all_files.append(os.path.join(root, file))
    return all_files

# Function to filter voice files into male and female categories
def filter_voice_files(files):
    male_files = []
    female_files = []
    
    for file in files:
        # Extract the actor number from the filename
        match = re.search(r"(\d{2})\.wav$", file)
        if match:
            actor_number = int(match.group(1))
            if actor_number % 2 == 0:  # Even numbers are female
                female_files.append(file)
            else:  # Odd numbers are male
                male_files.append(file)
                
    return male_files, female_files

# Function to randomly select voice files for characters
def select_voice_files_for_characters(characters):
    all_files = get_voice_files(voice_bank)
    male_files, female_files = filter_voice_files(all_files)

    selected_files = {}
    used_files = set()  # To keep track of used voice files

    for character in characters:
        if character in male_characters:
            # Select a random male voice that hasn't been used
            available_male_files = list(set(male_files) - used_files)
            if available_male_files:
                selected_files[character] = random.choice(available_male_files)
                used_files.add(selected_files[character])
        else:
            # Select a random female voice that hasn't been used, if available
            available_female_files = list(set(female_files) - used_files)
            if available_female_files:
                selected_files[character] = random.choice(available_female_files)
                used_files.add(selected_files[character])
            else:
                # Select any remaining file for other characters, ensuring it's not already used
                available_files = list(set(all_files) - used_files)
                if available_files:
                    selected_files[character] = random.choice(available_files)
                    used_files.add(selected_files[character])
    
    return selected_files

# Function to convert text to speech for a character
def voice_character(character, text, page_number, bubble_number, selected_files, save_directory):
    speaker_wav = selected_files.get(character)

    if speaker_wav:
        audio_output_filename = name_format.format(int(page_number), 0, bubble_number+1, ".wav")
        output_filename = os.path.join(save_directory, audio_output_filename)
        output = tts.tts_to_file(text=text, speaker_wav=speaker_wav, language="en")
        os.rename("output.wav", output_filename)  # Rename the default output file
        return output_filename
    else:
        raise ValueError(f"Character '{character}' not found in voice mapping.")

# Function to process the transcript and create audio files
def text2speech(pages, selected_files, save_directory):
    output_files = []
    time_file = []
    
    for page in pages: 
        page_number = page["page"]
        for bubble_number, (character, dialogue) in enumerate(page["lines"]):
            try:
                output = voice_character(character, dialogue, page_number, bubble_number, selected_files, save_directory)
                output_files.append(output)
            except ValueError as e:
                print(e)

    return output_files

In [ ]:
# Define the directory where the RAVDESS files are located
voice_bank = "/kaggle/input/ravdess-emotional-speech-audio/"
# Define male characters for the check
male_characters = ['teacher']
transcript_file = f"{transcript_path}/transcript.txt"

save_directory = audio_path

if os.path.exists(save_directory):
    shutil.rmtree(save_directory)
os.makedirs(save_directory, exist_ok=True)

# Get unique characters from the transcript
pages = parse_transcript(transcript_file, line_end = 20) # delete line_end to get audio of all page
characters = {line[0] for page in pages for line in page["lines"]}  # Get unique characters

# Select voice files for characters
selected_files = select_voice_files_for_characters(characters)

# # Call the function to process the transcript and generate audio files
output_files = text2speech(pages, selected_files, save_directory)